In [1]:
import numpy as np
import pandas as pd

In [2]:
company_x = [line.rstrip('\n') for line in open('company')]
apple_x = [line.rstrip('\n') for line in open('fruit')]

In [15]:
x=[]
x.extend(company_x)
x.extend(apple_x)

In [4]:
y=[]
for i in range(len(company_x)):
    y.append(1)
for i in range(len(apple_x)):
    y.append(0)

In [8]:
#clean text
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
REPLACE_BY_SPACE_RE=re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/vipss/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def clean_text(text):
    """
        text:a string
        return: clean string
    """
    for i in range(len(text)):
        text[i]=text[i].lower()
        text[i]=REPLACE_BY_SPACE_RE.sub(' ', text[i])#replace bad ones by extra spaces.
        text[i]=BAD_SYMBOLS_RE.sub('', text[i])#replace numbers and symbols by nothing
        text[i] = ' '.join(word for word in text[i].split() if word not in STOPWORDS)#remove stopwords
    return text

In [17]:
x=clean_text(x)

In [20]:
#max num of words
MAX_NB_WORDS=40000
#maximum words in each question
MAX_SEQUENCE_LENGTH=250
#embedding
EMBEDDING_DIM=100

In [21]:
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(x)
words=tokenizer.word_index

/home/vipss/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
from keras.preprocessing.sequence import pad_sequences
x=tokenizer.texts_to_sequences(x)
x=pad_sequences(x, MAX_SEQUENCE_LENGTH)

In [25]:
from sklearn.preprocessing import StandardScaler
def scl(tr, te):
    sc=StandardScaler()
    tr=sc.fit_transform(tr)
    te=sc.transform(te)
    return (tr,te, sc)

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.075, random_state=84)
x_train, x_test, scl=scl(x_train, x_test)

/home/vipss/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/vipss/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/vipss/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [33]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, SpatialDropout1D, Embedding
from keras.callbacks import EarlyStopping
model=Sequential()

In [35]:
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=x_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(output_dim=1, activation="sigmoid"))
model.compile(metrics=['accuracy'], loss="binary_crossentropy", optimizer="adam")
testing=model.fit(x_train, y_train, epochs=10, batch_size=10, validation_split=0.01, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

/home/vipss/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  after removing the cwd from sys.path.


Train on 392 samples, validate on 4 samples
Epoch 1/10
392/392 [==============================] - 9s 23ms/step - loss: 0.6575 - acc: 0.6071 - val_loss: 0.8078 - val_acc: 0.2500
Epoch 2/10
392/392 [==============================] - 7s 18ms/step - loss: 0.5585 - acc: 0.7219 - val_loss: 0.6104 - val_acc: 0.5000
Epoch 3/10
392/392 [==============================] - 7s 18ms/step - loss: 0.4663 - acc: 0.7806 - val_loss: 0.6932 - val_acc: 0.2500
Epoch 4/10
392/392 [==============================] - 7s 18ms/step - loss: 0.4084 - acc: 0.7985 - val_loss: 0.7500 - val_acc: 0.2500
Epoch 5/10
392/392 [==============================] - 7s 18ms/step - loss: 0.3812 - acc: 0.8010 - val_loss: 0.6410 - val_acc: 0.2500


In [36]:
accr=model.evaluate(x_test, y_test)

33/33 [==============================] - 0s 3ms/step


In [40]:
final_sc=StandardScaler()
x_final=final_sc.fit_transform(x)

/home/vipss/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/vipss/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [44]:
final_model=Sequential()
final_model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=x_final.shape[1]))
final_model.add(SpatialDropout1D(0.2))
final_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
final_model.add(Dense(output_dim=1, activation="sigmoid"))
final_model.compile(metrics=['accuracy'], loss="binary_crossentropy", optimizer="adam")
testing=final_model.fit(x_final, y, epochs=10, batch_size=10, validation_split=0.01, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

/home/vipss/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  """


Train on 424 samples, validate on 5 samples
Epoch 1/10
424/424 [==============================] - 9s 21ms/step - loss: 0.6591 - acc: 0.6250 - val_loss: 0.8124 - val_acc: 0.0000e+00
Epoch 2/10
424/424 [==============================] - 8s 18ms/step - loss: 0.5745 - acc: 0.6910 - val_loss: 0.7007 - val_acc: 0.6000
Epoch 3/10
424/424 [==============================] - 8s 18ms/step - loss: 0.4558 - acc: 0.7877 - val_loss: 0.4864 - val_acc: 0.8000
Epoch 4/10
424/424 [==============================] - 8s 18ms/step - loss: 0.4084 - acc: 0.7712 - val_loss: 0.5357 - val_acc: 0.8000
Epoch 5/10
424/424 [==============================] - 8s 18ms/step - loss: 0.3891 - acc: 0.7712 - val_loss: 0.5874 - val_acc: 0.6000
Epoch 6/10
424/424 [==============================] - 8s 19ms/step - loss: 0.3618 - acc: 0.8042 - val_loss: 0.3830 - val_acc: 0.6000
Epoch 7/10
424/424 [==============================] - 8s 18ms/step - loss: 0.3461 - acc: 0.8231 - val_loss: 0.2806 - val_acc: 1.0000
Epoch 8/10
424/424 [=

In [42]:
x_final.shape

(429, 250)